In [ ]:
! gdown --id 1EmzFdlodTARBOC16zVq7NAZNNeVDocWa

In [ ]:
! gdown --id https://drive.google.com/file/d/19-vrNVBYXsiuZ2km4-_FrO7imX1aUySH/view?usp=sharing

In [ ]:
! gdown --id 19-vrNVBYXsiuZ2km4-_FrO7imX1aUySH

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [ ]:
df_fake = pd.read_csv('/content/Fake.csv')
df_true = pd.read_csv('/content/True.csv')

In [ ]:
df_fake.head(5)   #to show first 5 results

In [ ]:
df_true.head(5)    #to show first 5 results



In [ ]:
df_fake["class"] = 0    #Inserting a column called "class" for fake and real news dataset to categories fake and true news.
df_true["class"] = 1


In [ ]:
df_fake.shape , df_true.shape


In [ ]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(22460,22450,-1):
    df_fake.drop([i], axis = 0, inplace = True)
df_true_manual_testing = df_true.tail(10)
for i in range(20403,20393,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [ ]:
df_fake.shape, df_true.shape

In [ ]:
df_fake_manual_testing["class"] = 0  #Merging the manual testing dataframe in single dataset and save it in a csv file
df_true_manual_testing["class"] = 1

In [ ]:
df_fake_manual_testing.head(10) #new fake manual testing data


In [ ]:
df_true_manual_testing.head(10)  #new true manual testing data


In [ ]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("/content/manual_testing.csv")

In [ ]:
df_marge = pd.concat([df_fake, df_true], axis =0 )  #Merging the main fake and true dataframe
df_marge.head(10)

In [ ]:
# @title date vs count()

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['date']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'date'}, axis=1)
              .sort_values('date', ascending=True))
  xs = counted['date']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df_marge.sort_values('date', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('subject')):
  _plot_series(series, series_name, i)
  fig.legend(title='subject', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('count()')

In [ ]:
# @title subject

from matplotlib import pyplot as plt
import seaborn as sns
df_marge.groupby('subject').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
df_marge.columns

In [ ]:
df = df_marge.drop(["title", "subject","date"], axis = 1) #"title", "subject" and "date" columns is not required for detecting the fake news, so dropping the columns.


In [ ]:
df.isnull().sum()  #eliminate null


In [ ]:
df = df.sample(frac = 1)  #randomly shuffling the data


In [ ]:
df.head() #desired dataset


In [ ]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [ ]:
df.columns

In [ ]:
df.head()


In [ ]:
def wordopt(text):  #Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
df["text"] = df["text"].apply(wordopt)


In [ ]:
x = df["text"]  #Defining dependent and independent variable as x and y
y = df["class"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25) #Splitting the dataset into training set and testing set.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  #convert text into vectors


In [ ]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [ ]:
#1. Logistic Regression
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

In [ ]:
pred_lr=LR.predict(xv_test)

In [ ]:
LR.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test, pred_lr))

In [ ]:
#2. Decision Tree Classification
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)


In [ ]:
pred_dt = DT.predict(xv_test)


In [ ]:
DT.score(xv_test, y_test)


In [ ]:
print(classification_report(y_test, pred_dt))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
#3. Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [ ]:
pred_gbc = GBC.predict(xv_test)

In [ ]:
GBC.score(xv_test, y_test)

0.9958103048671777

In [ ]:
print(classification_report(y_test, pred_gbc))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5851
           1       0.99      1.00      1.00      5367

    accuracy                           1.00     11218
   macro avg       1.00      1.00      1.00     11218
weighted avg       1.00      1.00      1.00     11218



In [ ]:
#4. Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
pred_rfc = RFC.predict(xv_test)

In [ ]:
RFC.score(xv_test, y_test)

0.9910857550365484

In [ ]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5851
           1       0.99      0.99      0.99      5367

    accuracy                           0.99     11218
   macro avg       0.99      0.99      0.99     11218
weighted avg       0.99      0.99      0.99     11218



In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                              output_lable(pred_DT[0]),
                                                                                                              output_lable(pred_GBC[0]),
                                                                                                              output_lable(pred_RFC[0])))

In [ ]:
news = str(input())
manual_testing(news)

21st Century Wire says This week, the historic international Iranian Nuclear Deal was punctuated by a two-way prisoner swap between Washington and Tehran, but it didn t end quite the way everyone expected. On the Iranian side, one of the U.S. citizens who was detained in Iran, Nosratollah Khosravi-Roodsari, has stayed in Iran, but on the U.S. side   all 7 of the Iranians held in U.S. prisons DID NOT show up to their flight to Geneva for the prisoner exchange   with at least 3 electing to stay in the U.S  TEHRAN SIDE: In Iran, 5 U.S. prisoners were released, with 4 of them making their way to Germany via Switzerland.Will Robinson Daily MailNone of the Iranians freed in the prisoner swap have returned home and could still be in the United States, it has been reported.The seven former inmates, who were released as part of a deal with the Islamic republic, did not show up to get a flight to Geneva, Switzerland, where the exchange was set to take place on Sunday.Three of the Iranians have d

In [ ]:
JAKARTA (Reuters) - Indonesia will buy 11 Sukhoi fighter jets worth $1.14 billion from Russia in exchange for cash and Indonesian commodities, two cabinet ministers said on Tuesday. The Southeast Asian country has pledged to ship up to $570 million worth of commodities in addition to cash to pay for the Suhkoi SU-35 fighter jets, which are expected to be delivered in stages starting in two years. Indonesian Trade Minister Enggartiasto Lukita said in a joint statement with Defence Minister Ryamizard Ryacudu that details of the type and volume of commodities were  still being negotiated . Previously he had said the exports could include palm oil, tea, and coffee. The deal is expected to be finalised soon between Indonesian state trading company PT Perusahaan Perdangangan Indonesia and Russian state conglomerate Rostec. Russia is currently facing a new round of U.S.-imposed trade sanctions. Meanwhile, Southeast Asia s largest economy is trying to promote its palm oil products amid threats of a cut in consumption by European Union countries. Indonesia is also trying to modernize its ageing air force after a string of military aviation accidents. Indonesia, which had a $411 million trade surplus with Russia in 2016, wants to expand bilateral cooperation in tourism, education, energy, technology and aviation among others.